In [1]:
import os
from openai import OpenAI

token = os.environ["GITHUB_TOKEN"]
# print(token)
# print(os.environ["OPENAI_API_KEY"])
endpoint = "https://models.inference.ai.azure.com"
model_name = "gpt-4o-mini"

client = OpenAI(
    base_url=endpoint,
    api_key=token,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "What is the capital of England?",
        }
    ],
    temperature=0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)


In [18]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

persist_directory = "docs/chroma2"
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [19]:
print(vectordb._collection.count())

150


In [20]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [21]:
from langchain_openai import OpenAI
llm = OpenAI(model="gpt-4o-mini")

### RetrievalQA

In [22]:
from langchain.chains import RetrievalQA

In [23]:
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever()
                                        )


In [25]:
result = qa_chain.invoke({"query": question})

In [26]:
result["result"]

' The major topics for this class include machine learning algorithms, probability, statistics, and linear algebra. Students will learn programming primarily in MATLAB or Octave and will also be introduced to concepts related to data networks, logistic regression, PCA, and more. The course aims to equip students with the skills to apply machine learning techniques to various problems and prepare them for potential research in the field.<|fim_suffix|>The major topics for this class include machine learning algorithms, probability, statistics, and linear algebra. Students will learn programming primarily in MATLAB or Octave and will also be introduced to concepts related to data networks, logistic regression, PCA, and more. The course aims to equip students with the skills to apply machine learning techniques to various problems and prepare them for potential research in the field.<|fim_suffix|>'

### Prompt